### Part1

The data is already in github.

In [48]:
import pandas as pd

### Part2

In [3]:
# read data from github
df = pd.read_csv("https://raw.githubusercontent.com/Tsun1/DAV-540/master/data.csv", index_col=[0, 1])
# reset the index
df.index = pd.MultiIndex.from_product([['ALASKA','AM WEST'], ['on time','delayed']])
df.index.names = ["airline", "type"]
# get full index values
df = df.swaplevel()
df = df.reset_index()
# to "long" format
df = pd.melt(df, ["airline", "type"])

In [4]:
df.head()

,airline,type,variable,value
0,ALASKA,on time,Los Angeles,497
1,ALASKA,delayed,Los Angeles,62
2,AM WEST,on time,Los Angeles,694
3,AM WEST,delayed,Los Angeles,117
4,ALASKA,on time,Phoenix,221


### Part 3

For each city, which airline had the best on time performance?

In [5]:
# get all unique cities and airlines
all_cities = df["variable"].unique()
all_airlines = df["airline"].unique()
for city in all_cities:
    performance = 0
    target = ""
    for airline in all_airlines:
        temp = df[(df["variable"]==city) & (df["airline"]==airline)]
        on_time = temp[temp["type"]=="on time"]["value"].values[0]
        delayed = temp[temp["type"]=="delayed"]["value"].values[0]
        temp_performance =  on_time / (on_time + delayed)
        # update the airline
        if temp_performance > performance:
            performance = on_time / (on_time + delayed)
            target = airline
    print(city, "best performance airline is", target)



Los Angeles best performance airline is ALASKA
Phoenix best performance airline is ALASKA
San Diengo best performance airline is ALASKA
San Francisco best performance airline is ALASKA
Seattle best performance airline is ALASKA


Which airline had the best overall on time performance?

In [6]:
best = ""
performance = 0
for airline in all_airlines:
    on_time = df[(df["type"] == "on time") & (df["airline"] == airline)]["value"].sum()
    delayed = df[(df["type"] == "delayed") & (df["airline"] == airline)]["value"].sum()
    # update the airline
    if on_time / (on_time + delayed) > performance:
        best = airline
        performance = on_time / (on_time + delayed)
print("The best performance airline is", best)

The best performance airline is AM WEST


### Part 4

In [49]:
# get on time and delayed dataframe 
temp1 =  df[df["type"]=="on time"].pivot(index="airline", columns="variable", values="value")
temp2 = df[df["type"]=="delayed"].pivot(index="airline", columns="variable", values="value")
temp1["type"] = "on time"
temp2["type"] = "delayed"
# merge two dataframes and set index
result = pd.concat([temp1, temp2]).reset_index().sort_values(by=["airline"]).set_index(["airline", "type"])
result

variable         Los Angeles  Phoenix  San Diengo  San Francisco  Seattle
airline type                                                             
ALASKA  on time          497      221         212            503     1841
        delayed           62       12          20            102      305
AM WEST on time          694     4840         383            320      201
        delayed          117      415          65            129       61